Converting an RTDS dataset for instance segmentation

In [ ]:
import json
import os
from os import listdir
from os.path import isfile, join
from shutil import copyfile

import pandas as pd

In [ ]:
ROAD_SIGN = "road sign"

# The minimum size of the image with the sign
IMAGE_SIZE_PX = 30

# Number of character images for each of the superclasses
IMAGES_PER_CLASS = {
    "1": 10,
    "2": 50,
    "3": 15,
    "4": 20,
    "5": 75,
    "7": 10,
    "8": 15,
}

# Full path to the dataset
RUSSIAN_ROAD_SIGN_DT = "rtds/full-frames"
IMAGE_DIR = os.path.join(RUSSIAN_ROAD_SIGN_DT, "rtsd-frames")

# Path to the converted dataset
DT_DIR = "dt"

In [ ]:
def drop_extension(filename):
    dot = filename.index(".")
    return filename[:dot]


def load_sign_classes():
    sign_icons_path = os.path.join(RUSSIAN_ROAD_SIGN_DT, "sign-icons")

    sign_classes = {}
    for superclass in listdir(sign_icons_path):
        superclass_folder = os.path.join(sign_icons_path, superclass)
        file_names = [drop_extension(f) for f in listdir(superclass_folder) if isfile(join(superclass_folder, f))]
        sign_classes[superclass] = file_names
    return sign_classes

sign_classes = load_sign_classes()

In [ ]:
def get_superclass(class_id):
    for superclass, classes in sign_classes.items():
        if class_id in classes:
            return superclass
    return None

# Returns the shape of a sign by its class
# Possible shape: ellipse, rectangle, inv_triangle, triangle, rhomb
def shape_from_class_id(class_id):
    superclass = get_superclass(class_id)
    if superclass is None:
        if class_id[0] == "3" or class_id[0] == "4":
            return "ellipse"
        else:
            return "rectangle"

    if superclass == "1":
        if class_id == "2_4":
            return "inv_triangle"
        else:
            return "triangle"
    if superclass == "2":
        return "rhomb"
    if superclass == "3":
        return "ellipse"
    if superclass == "4":
        if class_id == "4_8_1" or class_id == "4_8_2" or class_id == "4_8_3":
            return "rectangle"
        else:
            return "ellipse"
    return "rectangle"

In [ ]:
def create_shape_attributes(sign):
    type = sign["sign_type"]
    x_from = sign["x_from"]
    y_from = sign["y_from"]
    width = sign["width"]
    height = sign["height"]

    if type is None:
        print(sign)
    if type == "ellipse":
        name = "ellipse"
    else:
        name = "polygon"
    shape_attributes = {"name": name}

    if type == 'ellipse':
        shape_attributes["cx"] = x_from + width / 2
        shape_attributes["cy"] = y_from + height / 2
        shape_attributes["rx"] = width / 2
        shape_attributes["ry"] = shape_attributes["rx"]
    elif type == 'rectangle':
        shape_attributes["all_points_x"] = [x_from, x_from + width, x_from + width, x_from, x_from]
        shape_attributes["all_points_y"] = [y_from, y_from, y_from + height, y_from + height, y_from]
    elif type == 'rhomb':
        shape_attributes["all_points_x"] = [x_from, x_from + width / 2, x_from + width, x_from + width / 2, x_from]
        shape_attributes["all_points_y"] = [y_from + height / 2, y_from, y_from + height / 2, y_from + height,
                                            y_from + height / 2]
    elif type == "triangle":
        shape_attributes["all_points_x"] = [x_from, x_from + width / 2, x_from + width, x_from]
        shape_attributes["all_points_y"] = [y_from + height, y_from, y_from + height, y_from + height]
    elif type == "inv_triangle":
        shape_attributes["all_points_x"] = [x_from, x_from + width, x_from + width / 2, x_from]
        shape_attributes["all_points_y"] = [y_from, y_from, y_from + height, y_from]
    else:
        raise Exception("Unknown type " + str(type))
    return shape_attributes


def make_region_data(shape_attributes):
    regions = {}
    for i in range(len(shape_attributes)):
        region_data = {
            "shape_attributes": shape_attributes[i],
            "region_attributes": {
                "name": ROAD_SIGN
            }
        }
        regions[i] = region_data
    return regions

In [ ]:
file_classes = {}

def any_interesting_signs(signs):
    for _, dt in signs.iterrows():
        file_class = dt['sign_class']
        
        width = dt['width']
        height = dt['height']
        
        class_idxs = file_class.split('_')
        class_id = class_idxs[0] + class_idxs[1]
        

        if not (class_id in file_classes):
            file_classes[class_id] = 0

        superclass = get_superclass(file_class)
        if superclass is None:
            superclass = class_idxs[0]
        if superclass == "6":
            superclass = "5"
        if width > IMAGE_SIZE_PX and height > IMAGE_SIZE_PX and file_classes[class_id] < IMAGES_PER_CLASS[superclass]:
            return True
    return False

In [ ]:

file_dt = pd.read_csv('full-gt.csv')

signs = {}
count = 0
for idx, sign_dt in file_dt.iterrows():
    filename = sign_dt['filename']
    if filename in signs:
        continue

    file_signs = file_dt[(file_dt["filename"] == filename)]

    if not (any_interesting_signs(file_signs)):
        continue

    for i, dt in file_signs.iterrows():
        file_class = dt['sign_class']
        words = file_class.split('_')
        class_id = words[0] + words[1]

        sign_type = shape_from_class_id(file_class)

        if not (class_id in file_classes):
            file_classes[class_id] = 0

        file_classes[class_id] += 1

        x_from = dt['x_from']
        y_from = dt['y_from']
        width = dt['width']
        height = dt['height']

        sign = {"sign_type": sign_type,
                    "x_from": x_from,
                    "y_from": y_from,
                    "width": width,
                    "height": height
                }
        if not (filename in signs):
                signs[filename] = []
        signs[filename].append(sign)

        count += 1
        print("Class: " + str(file_class) + ", Filename: " + filename)

print("Total count " + str(count))

file_index = 1

via_region_data = {}

for filename, sign in signs.items():
    file_index += 1

    file_path = os.path.join(IMAGE_DIR, filename)

    annotation = {
            "filename": str(file_index) + ".jpg",
            "fileref": "",
            "size": os.path.getsize(file_path),
            "base64_img_data": "",
            "file_attributes": {},
            "regions": make_region_data(list(map(create_shape_attributes, sign)))
    }

    copyfile(file_path, os.path.join(DT_DIR, annotation["filename"]))

    annotation_name = annotation["filename"] + str(annotation["size"])

    via_region_data[annotation_name] = annotation

with open("via_region_data.json", "w") as write_file:
    json.dump(via_region_data, write_file)